In [ ]:
import requests
import os
from pathlib import Path

def progress_bar(some_iter):
    try:
        from tqdm import tqdm
        return tqdm(some_iter)
    except ModuleNotFoundError:
        return some_iter

def download_file_from_google_drive(id, destination):
    print("Trying to fetch {}".format(destination))

    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        if not os.path.exists(os.path.split(destination)[0]):
            path = Path(os.path.split(destination)[0])
            path.mkdir(parents=True)

        with open(destination, "wb") as f:
            for chunk in progress_bar(response.iter_content(CHUNK_SIZE)):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [ ]:
download_file_from_google_drive("1xeUnqkhuzGGzZKThzPeXe2Vf6Uu_g_xM", "./data/yelp/raw_train.csv")
download_file_from_google_drive('1G42LXv72DrhK4QKJoFhabVL4IU6v2ZvB', './data/yelp/raw_test.csv')